## Extending a RIOT application

In this exercise, you will extend a RIOT application with new shell commands and add an interactive behavior:

- the 'board' command will return the name of the board
- the 'cpu' command will return the name of the cpu
- the 'temperature' command will return the current temperature measured by the on-board sensor

For all shell commands, this notebook will precisely give you though the implementation and give you all useful documentation links (online documentation and similar sample applications).

### Add the shell module to the build

Edit the file `Makefile` and add there the `shell` module to the build.

```mk
USEMODULE += shell
```

### Add basic shell to the application

The first thing to do is to include the `shell.h` header at the beginning of the `main.c` file, just below the line `#include <stdio.h>`:

```c
#include "shell.h"
```

Then, in the `main` function, the shell can be started by adding the following code before `return 0`:

```c
    char line_buf[SHELL_DEFAULT_BUFSIZE];
    shell_run(NULL, line_buf, SHELL_DEFAULT_BUFSIZE);
```

The application can be tested on the `native` platform with the following commands (**run them in the separated terminal on the right**):

```sh
make -C riot/basics/shell all term
```

By default, the shell module provides the `help` command to list available commands. Try it.

As you can see, there is no available command.

**Important:** Use Ctrl+C to quit the RIOT native instance running in the terminal.

Let's implement a couple of simple ones to print the board and cpu.

### Implement simple shell commands

RIOT provides 2 predefined macros that contain the current board and cpu the application is running on: `RIOT_BOARD` and `RIOT_CPU`. You will use them to print the board and cpu in the corresponding shell commands, e.g. `board` and `cpu` callbacks.

1. In `main.c`, let's add the callback functions for the `board` and `cpu` commands, just below the line `#include "shell.h`:

```c
static int _board_handler(int argc, char **argv)
{
    (void)argc; /* This parameter is not used, avoid a warning during build */
    (void)argv;
    
    puts(RIOT_BOARD);
    
    return 0;
}

static int _cpu_handler(int argc, char **argv)
{
    (void)argc; /* This parameter is not used, avoid a warning during build */
    (void)argv;
    
    puts(RIOT_CPU);
    
    return 0;
}
```

2. Now let's define the command name, the help message and the associated callback function in the list of available shell commands. This is simply done by adding the following code between the function callbacks and the main function:

```c
static const shell_command_t shell_commands[] = {
    { "board", "Print the board name", _board_handler },
    { "cpu", "Print the cpu name", _cpu_handler },
    { NULL, NULL, NULL }
};
```

3. In the main function, modify the `shell_run` call to integrate the new extended `shell_commands` list:

```c
shell_run(shell_commands, line_buf, SHELL_DEFAULT_BUFSIZE);
```

instead of:
```c
shell_run(NULL, line_buf, SHELL_DEFAULT_BUFSIZE);
```

4. In the terminal on the right, rebuild and run the application for the native board:

```sh
make -C riot/basics/shell all term
```


### Run on iotlab-m3

The application is now ready and can be tested on real hardware, on an iotlab-m3 in the IoT-LAB testbed.

1. Submit an experiment with one M3 device:

In [ ]:
!iotlab-experiment submit -n "riot-shell" -d 20 -l 1,archi=m3:at86rf231+site=grenoble

In [ ]:
!iotlab-experiment wait

In [ ]:
!iotlab-experiment get -r

2. Build and flash the shell application for the iotlab-m3 target device:

**Note:** change the `<id>` value set to the `IOTLAB_NODE` variable with the m3 device id associated to your experiment (from the `iotlab-experiment get -r` command above).

In [ ]:
!make BOARD=iotlab-m3 IOTLAB_NODE=m3-<id>.grenoble.iot-lab.info flash

3. Open a serial terminal remotely in the Jupyter terminal on the right:

```sh
make BOARD=iotlab-m3 IOTLAB_NODE=m3-<id>.grenoble.iot-lab.info -C riot/basics/shell term
```